In [ ]:
import json
# make sure to install these packages before running:
# !pip install pandas
# !pip install sodapy

import pandas as pd
from sodapy import Socrata

# Example authenticated client (needed for non-public datasets):
client = Socrata("opendata.utah.gov",
                 "b8AQz7fuwpqdLHiNzouIy8BOl",
                 username="alicialane01@gmail.com",
                 password="d00A@EJ3Ab^ac#J")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("herb-zqda", limit=300000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_dict(results)

In [ ]:
#Use this to reset df values/columns without having to rerun previous block of code
collision_df = results_df.copy() 
collision_df.shape

In [ ]:
#convert columns to correct data type as described in the data dictionary
import numpy as np

collision_df = collision_df.astype({
    'crash_datetime': np.datetime64,
    # 'route': np.string_,
    'milepoint': np.number,
    'lat_utm_y': np.number,
    'long_utm_x': np.number,
    # 'main_road_name': np.string_,
    # 'city': np.string_,
    # 'county_name': np.string_,
    'crash_severity_id': np.number,
    'work_zone_related': np.bool_,
    'pedestrian_involved': np.bool_,
    'bicyclist_involved': np.bool_,
    'motorcycle_involved': np.bool_,
    'improper_restraint': np.bool_,
    'unrestrained': np.bool_,
    'dui': np.bool_,
    'intersection_related': np.bool_,
    'wild_animal_related': np.bool_,
    'domestic_animal_related': np.bool_,
    'overturn_rollover': np.bool_,
    'commercial_motor_veh_involved': np.bool_,
    'teenage_driver_involved': np.bool_,
    'older_driver_involved': np.bool_,
    'night_dark_condition': np.bool_,
    'single_vehicle': np.bool_,
    'distracted_driving': np.bool_,
    'drowsy_driving': np.bool_,
    'roadway_departure': np.bool_ 
})
collision_df.dtypes

In [ ]:
collision_df.isna().sum()

In [ ]:
#handle missing data (there are no columns with 50% or more missing data so we kept all columns)
def handle_missing(df, percent = 0.5):
  for col in df:
    if df[col].isna().sum()/len(df) > percent:
      df.drop(columns = [col], inplace = True)

      print('Dropped ', col)

  return df

handle_missing(collision_df, 0.5)
collision_df.shape[1]
    

In [ ]:
#Use to bin variables (maybe exclude columns like Street Name that don't have clear categorical values)

# def bin_variables(df, percent = 0.05):
#   for col in df:
#     if not pd.api.types.is_numeric_dtype(df[col]):
#       for value, count in df[col].value_counts().iteritems():
#         if count/ len(df) < percent:
#           df.loc[df[col] == value, col] = 'Other'

#   return df

# bin_variables(df)
# df['MSZoning'].value_counts()

In [ ]:
# add columns for day of year, day of week, month, time of date, year, 
collision_df['crash_date'] = pd.to_datetime(collision_df['crash_datetime']).dt.date
collision_df['crash_time'] = pd.to_datetime(collision_df['crash_datetime']).dt.time

In [ ]:
df = collision_df[collision_df['city'] == False]
df